<a href="https://colab.research.google.com/github/skywalker0803r/deep-learning-ian-goodfellow/blob/master/%E6%99%82%E9%96%93%E5%BA%8F%E5%88%97%E6%A8%A1%E6%9D%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import Adam
import pandas as pd
from torch.utils.data import TensorDataset,DataLoader

class R2D2(nn.Module):
  def __init__(self,obs_size,n_actions,hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.l1 = nn.Linear(obs_size,self.hidden_size)
    self.l2 = nn.LSTMCell(self.hidden_size,self.hidden_size)
    self.l3 = nn.Linear(self.hidden_size,n_actions)

  def forward(self,x,hx,cx):
    x = torch.tanh(self.l1(x))
    hx,cx = self.l2(x,(hx,cx))
    x = torch.tanh(hx)
    x = torch.sigmoid(self.l3(x)) # to range[0,1]
    return x,hx,cx

In [25]:
def r2d2_forward(model,x,eval=False,time_step=3):
  # 切換模式
  if eval == True:
    model.eval()
  else:
    model.train()

  # 初始化隱變量
  hx = torch.zeros((x.size()[0],model.hidden_size))
  cx = torch.zeros((x.size()[0],model.hidden_size))

  # 空Y
  values = torch.FloatTensor()

  # RNN FORWARD用法
  for t in range(time_step):
    # 根據當前t時刻狀態,動作,hx,cx當輸入得到,v(產出),更新hx,cx
    v,hx,cx = model(x[:,t,:],hx,cx)
    values = torch.cat((values,v),dim=1)

  # 一系列產出(Batch,Time,Features)
  values = values.reshape(x.size()[0],time_step,-1)

  # 只取最後一個時間點
  return values[:,-1,:] # return last time_step

In [26]:
model = R2D2(
    obs_size=5,
    n_actions=1,
    hidden_size=64
    )

# BATCH , TIME ,FEATURE

In [27]:
x = torch.rand(64,16,5)
x.shape

torch.Size([64, 16, 5])

# BATCH,FEATURE

In [28]:
y = r2d2_forward(model,x,eval=False)
y.shape

torch.Size([64, 1])